In [1]:
import numpy as np
import pandas as pd
import os
import math
from scipy.fft import fft
from tqdm import tqdm

### Creación de los archivos de energía espectral con 512 valores

In [2]:
# lat: latitud de la localización en la que se registraton los datos
# dirData es la ruta al directorio en el que está los datos
# file: nombre del archivo de datos a procesar
# startTime: fecha de comienzo de las medidas -> 'yyyy-mm-dd hh:mm:ss'

def crear_espectro(lat, dirData, file, startTime):

    # SET PARAMETERS
    lat = lat # latitude (degrees)
    Fs = 1 # sampling frequency (Hz)
    T = 1/Fs # sampling period (s)
    L = 1024 # number of samples in a single burst = time vector length
    t = L*T # burst length (s)

    os.chdir(dirData)
    data = pd.read_csv(file, header = None) 
    data.columns = ['Time', 'Burst', 'Pressure']
    data['Time'] = pd.to_datetime(data['Time'])

    StartTime = pd.to_datetime(startTime)
    AirPressureFile = 'airpressure.txt' # change accordingly
    AirPressureSeries = pd.read_csv(AirPressureFile, header = None)

    # PREPARE TABLES TO SAVE OUTPUTS
    spectra = pd.DataFrame(index = range(data['Burst'].max()), columns = range(1,int(L/2)+1)) #From 1 to exclude DC
    spectraName = file[:-4] + '_spectra.txt'

    # PROCESS DATA ON THE BURST-BY-BURST BASIS
    for n in tqdm(range(min(data['Burst']), max(data['Burst']) + 1), desc="Procesando Bursts"):
        df = data[data['Burst'] == n]
        df = df.reset_index(drop = True)
        df_time = df.Time[0]
        TimeDifference = df.Time[0] - StartTime
        TimeDifference = int(TimeDifference.total_seconds()/3600) # (h)
        AirPressure = AirPressureSeries.loc[TimeDifference, 0]

        # CALCULATE SEA PRESSURE
        df['SeaPressure'] = pd.Series(np.zeros(df.shape[0] + 1))
        df['SeaPressure'] = df['Pressure'] - (AirPressure/100) # mbar2dbar

        # CALCULATE DEPTH FROM PRESSURE 
        # UNESCO formula (Fofonoff & Millard, 1983) under the assumption of 
        # constant water temperature 0 C and salinity 35 PSU
        x = math.sin(lat/57.29578)**2
        g = 9.780318*(
            1+(5.2788*10**-3+2.36*10**-5*x)
            *x)+1.092*10**-6*df['SeaPressure'] # acceleration due to gravity (m/s2)
        df['Depth'] = pd.Series(np.zeros(df.shape[0] + 1))
        df['Depth'] = ((((
            -1.82*10**-15*df['SeaPressure']+2.279*10**-10)
            *df['SeaPressure']-2.2512*10**-5)
            *df['SeaPressure']+9.72659)
            *df['SeaPressure'])/g # (m)

        # REMOVE THE SLOWLY-VARYING COMPONENT OF WATER DEPTH
        # (effect of e.g. sea level rise, tides and storm surges)
        # Subtract a fitted 2nd order polynomial trend from each burst
        time = range(L) # time vector [0:1:1023]
        tides = np.polyfit(time, df['Depth'], 2)
        S1 = df['Depth'] - np.polyval(tides, time) # depth variability associated with wind waves (m)

        # CALCULATE WAVE SPECTRA
        S1 = S1.to_numpy()
        Y = fft(S1) # Fast Fourier Transform
        P2 = abs(Y)
        #Include DC Component
        # P1 = P2[:int(L/2+1)]**2/(Fs*L)
        # P1[1:-1] = 2*P1[1:-1] # energy density spectrum at depth (m2 s)
        # f = np.r_[:int(L/2+1)]*Fs/L # frequency vector [0:d_f:0.5]
        # d_f = f[1] - f[0] # frequency interval
        #Exclude DC Component
        P1 = P2[1:int(L/2+1)]**2 / (Fs * L)
        P1[1:-1] = 2*P1[1:-1] # energy density spectrum at depth (m2 s)
        f = np.r_[1:int(L/2+1)]*Fs/L # frequency vector [0:d_f:0.5]
        d_f = f[1] - f[0] # frequency interval

        # APPLY DEPTH CORRECTION
        h = df['Depth'].mean() # mean depth (m)
        z = df['Depth'].mean() # logger depth (m)
        g = 9.780318*(
            1+(5.2788*10**-3+2.36*10**-5*x)
            *x)+1.092*10**-6 # acceleration due to gravity (m/s2)
        k = np.linspace(0,1000,100001) # basic wavenumber values [0:0.01:1000] (1/m)
        fA = (g*k*np.tanh(k*h))**0.5/(2*math.pi) # corresponding basic wave frequencies (Hz)
        A = np.cosh(k*(h-z))/np.cosh(k*h) # set of correction factors
        Af = np.interp(f, fA, A) # correction factor interpolated to the burst frequencies
        Ag = 0.001 # attenuation cut-off value
        Af[Af<Ag] = np.nan
        P1_A = P1/Af # energy density spectrum at the sea surface (m2 s)
        #
        # # ADD A HIGH FREQUENCY TAIL
        # # E(f)~f^(-4) after Kaihatu et al., 2007
        lastval = (np.isnan(P1_A)).argmax()-1 # highest frequency for which Af>Ag
        pom1 = sum(P1_A[lastval-9:lastval+1]*f[lastval-9:lastval+1]**-4)
        pom2 = sum(f[lastval-9:lastval+1]**-8)
        Amp = pom1/pom2
        P1_A[lastval+1:] = Amp*f[lastval+1:]**-4
        #
        # # SAVE OUTPUTS
        #spectra.loc[n-1,:] = P1_A.round(6)
        spectra.loc[n-1,:] = P1_A.round(6)

    spectra.to_csv(spectraName, index = False)

In [15]:
crear_espectro(lat = 76.9952, dirData='/home/adrian.perez/Documents/OceanWave/HDP-GPC/hdpgpc/data/ocean/', file='VES3.txt', startTime = '2019-06-25 00:00:00')

Procesando Bursts:   0%|          | 0/14376 [00:00<?, ?it/s]/tmp/ipykernel_2531943/1108223764.py:85: RuntimeWarning: overflow encountered in cosh
  A = np.cosh(k*(h-z))/np.cosh(k*h) # set of correction factors
Procesando Bursts:   0%|          | 1/14376 [17:10<4115:15:43, 1030.60s/it]


KeyboardInterrupt: 

In [4]:
# Calculo del vector de frecuencias
fm = 1 # Frecuencia de muestreo de un Hz
N = 1024 # número de muestras

freq = np.linspace(1/1024, fm/2, N//2+1)
freq = freq.reshape(-1, 3).mean(axis=1)

np.save('../data/freq.npy', freq)

FileNotFoundError: [Errno 2] No such file or directory: '../data/freq.npy'

In [8]:
data = pd.read_csv("../data/RawData/VES2_spectra.txt").drop(columns=['512'])

data = data.T.groupby(np.arange(len(data.columns)) // 3).mean()
data = data.rolling(window=2).mean()
data = data.fillna(0)
data = data.rolling(window=2).mean()
data = data.fillna(0)
data = data.T
data = data.values[:,:,np.newaxis]

np.save('../data/VES2.npy', data)